In [1]:
import os
import sys

module_path = os.path.abspath("..")
if module_path not in sys.path:
    sys.path.append(module_path)

import tensorflow as tf
from tensorflow.keras import backend as K
from interactions import ExactInteractions

import numpy as np


## Exact layer 

Creates a interaction matrix S were each entry $i$ and $j$ are defined by:

$s_{ij}=\begin{cases}1 & q_i = d_i\\0 & otherwise\end{cases}$

In [2]:
   
K.clear_session()

input_query = tf.keras.layers.Input((10,), dtype="int32")
input_sentence = tf.keras.layers.Input((12,), dtype="int32")

exact_interaction = ExactInteractions()

_out = exact_interaction([input_query, input_sentence])

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()
    

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12)]         0                                            
__________________________________________________________________________________________________
exact_interactions (ExactIntera (None, 10, 12)       0           input_1[0][0]                    
                                                                 input_2[0][0]                    
Total params: 0
Trainable params: 0
Non-trainable params: 0
__________________________________________________________________________________________________


In [3]:
query = np.random.randint(1,4,(1, 10))
document = np.random.randint(1,4,(1, 12))

In [4]:
y = model.predict([query, document])
y

array([[[0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]]], dtype=float32)

In [5]:
# add padding
query[:,5:] = np.zeros(5,)
document[:,6:] = np.zeros(6,)
y = model.predict([query, document])
y

array([[[0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

# Exact Layer with term importance

In [6]:
   
K.clear_session()

input_query = tf.keras.layers.Input((8,), dtype="int32")
input_sentence = tf.keras.layers.Input((4,), dtype="int32")

input_query_t_importance = tf.keras.layers.Input((8,), dtype="float32")
input_sentence_t_importance = tf.keras.layers.Input((4,), dtype="float32")

_inputs = [input_query, input_sentence, input_query_t_importance, input_sentence_t_importance]

exact_interaction = ExactInteractions()

_out = exact_interaction(_inputs)

model = tf.keras.models.Model(inputs=_inputs, outputs=_out)
model.summary()
    

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 4)]          0                                            
______________________________________________________________________________________________

In [20]:
query = np.random.randint(1,10,(1, 8))
sentence = np.random.randint(1,10,(1, 4))

query_importance = np.random.random((1, 8))
sentence_importance = np.random.random((1, 4))

In [21]:
y = model.predict([query, sentence, query_importance, sentence_importance])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [1. 0. 0. 0.]]
[[0.0992586  0.0992586  0.0992586  0.0992586 ]
 [0.8366827  0.8366827  0.8366827  0.8366827 ]
 [0.552322   0.552322   0.552322   0.552322  ]
 [0.18749177 0.18749177 0.18749177 0.18749177]
 [0.9671816  0.9671816  0.9671816  0.9671816 ]
 [0.52295625 0.52295625 0.52295625 0.52295625]
 [0.84259874 0.84259874 0.84259874 0.84259874]
 [0.20079795 0.20079795 0.20079795 0.20079795]]
[[0.8589076  0.90894186 0.2348578  0.8571401 ]
 [0.8589076  0.90894186 0.2348578  0.8571401 ]
 [0.8589076  0.90894186 0.2348578  0.8571401 ]
 [0.8589076  0.90894186 0.2348578  0.8571401 ]
 [0.8589076  0.90894186 0.2348578  0.8571401 ]
 [0.8589076  0.90894186 0.2348578  0.8571401 ]
 [0.8589076  0.90894186 0.2348578  0.8571401 ]
 [0.8589076  0.90894186 0.2348578  0.8571401 ]]


In [22]:
query[:,4:] = np.zeros(4,)
sentence[:,2:] = np.zeros(2,)
query_importance[:,4:] = np.zeros(4,)
sentence_importance[:,2:] = np.zeros(2,)

y = model.predict([query, sentence, query_importance, sentence_importance])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0.0992586  0.0992586  0.         0.        ]
 [0.8366827  0.8366827  0.         0.        ]
 [0.552322   0.552322   0.         0.        ]
 [0.18749177 0.18749177 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
[[0.8589076  0.90894186 0.         0.        ]
 [0.8589076  0.90894186 0.         0.        ]
 [0.8589076  0.90894186 0.         0.        ]
 [0.8589076  0.90894186 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]


# Semantic Layer

In [23]:
from interactions import SemanticInteractions

In [24]:
from nir.embeddings import FastText
from nir.tokenizers import Regex

cache_folder = "/backup/IR"
prefix_name = "disk4_5"

# load tokenizer
tk = Regex.load_from_json(cache_folder=cache_folder, prefix_name=prefix_name)

# load embedding matrix
ft = FastText.maybe_load(cache_folder = cache_folder,
                         prefix_name = prefix_name,
                         path = "/backup/pre-trained_embeddings/fasttext/wiki.en.bin",
                         tokenizer = tk)

emb_matrix = ft.embedding_matrix()

DEBUG created tokenizer disk4_5_RegexTokenizer
False False
[LOAD FROM CACHE] Load embedding matrix from /backup/IR/embedding_wiki_disk4_5_RegexTokenizer


In [25]:

K.clear_session()

input_query = tf.keras.layers.Input((8,), dtype="int32")
input_sentence = tf.keras.layers.Input((4,), dtype="int32")

semantic_interaction = SemanticInteractions(emb_matrix)

_out = semantic_interaction([input_query, input_sentence])

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()
    

[EMBEDDING MATRIX SHAPE] (228107, 300)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
semantic_interactions (Semantic (None, 8, 4, 3)      600         input_1[0][0]                    
                                                                 input_2[0][0]                    
Total params: 600
Trainable params: 600
Non-trainable params: 0
__________________________________________________________________________________________

In [26]:
query = np.random.randint(1,100000,(1, 8))
document = np.random.randint(1,100000,(1, 4))

In [27]:
y = model.predict([query, document])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[0.12195162 0.3060188  0.07169754 0.22878772]
 [0.15432015 0.3396184  0.05373091 0.31086975]
 [0.15998912 0.22292125 0.09267265 0.23756436]
 [0.15174542 0.22771084 0.14356448 0.10540374]
 [0.15426344 0.08843064 0.17623109 0.20108868]
 [0.20685682 0.257168   0.08523287 0.24533218]
 [0.12268002 0.24191794 0.05046202 0.19189438]
 [0.22307785 0.08852232 0.10685393 0.19338246]]
[[-9.8678552e-02 -9.8678552e-02 -9.8678552e-02 -9.8678552e-02]
 [ 2.5561264e-02  2.5561264e-02  2.5561264e-02  2.5561264e-02]
 [ 4.4735774e-02  4.4735774e-02  4.4735774e-02  4.4735774e-02]
 [-9.1937594e-03 -9.1937594e-03 -9.1937594e-03 -9.1937594e-03]
 [-3.7848368e-02 -3.7848368e-02 -3.7848368e-02 -3.7848368e-02]
 [-3.4315750e-02 -3.4315750e-02 -3.4315750e-02 -3.4315750e-02]
 [ 7.3319417e-05  7.3319417e-05  7.3319417e-05  7.3319417e-05]
 [ 1.7905701e-02  1.7905701e-02  1.7905701e-02  1.7905701e-02]]
[[-6.1611237e-05  2.0398768e-02  4.9427930e-02 -8.2756080e-02]
 [-6.1611237e-05  2.0398768e-02  4.9427930e-02 -8.27560

In [28]:
# add padding
query[:,4:] = np.zeros(4,)
document[:,2:] = np.zeros(2,)
y = model.predict([query, document])
print(y[0,:,:,0])
print(y[0,:,:,1])
print(y[0,:,:,2])

[[ 0.12195162  0.3060188  -0.         -0.        ]
 [ 0.15432015  0.3396184  -0.         -0.        ]
 [ 0.15998912  0.22292125 -0.         -0.        ]
 [ 0.15174542  0.22771084  0.          0.        ]
 [-0.         -0.          0.          0.        ]
 [-0.         -0.          0.          0.        ]
 [-0.         -0.          0.          0.        ]
 [-0.         -0.          0.          0.        ]]
[[-0.09867855 -0.09867855 -0.         -0.        ]
 [ 0.02556126  0.02556126  0.          0.        ]
 [ 0.04473577  0.04473577  0.          0.        ]
 [-0.00919376 -0.00919376 -0.         -0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
[[-6.1611237e-05  2.0398768e-02 -0.0000000e+00 -0.0000000e+00]
 [-6.1611237e-05  2.0398768e-02 -0.0000000e+00 -0.0000000e+00]
 [-6.1611237e-05  2.0398768e-02 -0.0000000e+00 -0.000000

# Context semantic layer

In [8]:
from interactions import MatrixInteractionMasking

class ContextedSemanticInteractions(MatrixInteractionMasking):
    
    def __init__(self, 
                 context_embedding_layer = None,
                 cls_token_id = None,
                 sep_token_id = None,
                 pad_token_id = None,
                 context_embedding_dim = None, # used to compute the term importance of each query and sentence token
                 initializer='glorot_uniform',
                 regularizer=None,
                 **kwargs):
        
        super(ContextedSemanticInteractions, self).__init__(**kwargs)

        self.context_embedding_layer = context_embedding_layer
        self.context_embedding_dim = context_embedding_dim
        self.cls_token_id = cls_token_id
        self.sep_token_id = sep_token_id
        self.pad_token_id = pad_token_id
        self.initializer = initializer
        self.regularizer = regularizer
        
    def build(self, input_shape):
        
        # add some weight that will learn term importance projection of the query and sentence embeddings
        if self.context_embedding_dim is not None:
            self.query_w = self.add_weight(name="query_w",
                                           shape=(self.context_embedding_dim, 1),
                                           initializer=self.initializer,
                                           regularizer=self.regularizer,
                                           trainable=True)
            
            self.sentence_w = self.add_weight(name="sentence_w",
                                              shape=(self.context_embedding_dim, 1),
                                              initializer=self.initializer,
                                              regularizer=self.regularizer,
                                              trainable=True)
        
        super(ContextedSemanticInteractions, self).build(input_shape)
        
    def _produce_context_embeddings(self, query_vector, sentence_vector):
        # assume transformer layer follows a BERT architecture
        assert(self.cls_token_id is not None and self.sep_token_id is not None and self.pad_token_id is not None)
        
        batch_dim = tf.shape(query_vector)[0]
        
        cls_input = K.expand_dims(tf.ones((batch_dim,))*tf.constant(self.cls_token_id))
        sep_input = K.expand_dims(tf.ones((batch_dim,))*tf.constant(self.sep_token_id))
        
        _input = K.concatenate([cls_input, query_vector, sep_input, sentence_vector, sep_input])

        _out = self.context_embedding_layer(_input)
        
        context_query = _out[:,1:self.query_max_elements+1,:]
        context_sentence = _out[:,self.query_max_elements+2:self.query_max_elements+2+self.setence_max_elements,:]
        
        return context_query, context_sentence
        
    def call(self, x, mask=None):
        """
        x[0] - padded query tokens id's
        x[1] - padded sentence tokens id's
        
        or 
        
        For this setting the mask is needed
        x[0] - padded query context embeddings
        x[1] - padded sentence context embeddings
        
        or
        
        For this setting the mask is needed
        x    - pre-computed similarity matrix, dims (B,Q,D)
        """
        
        if mask is None:
            mask = self.compute_mask(x)
        
        if self.context_embedding_layer is not None:
            query_context_embeddings, sentence_context_embeddings = self._produce_context_embeddings(x[0], x[1]) 
        else:
            query_context_embeddings, sentence_context_embeddings = (x[0],x[1])
            
        if len(x)==2:
            interaction_matrix = tf.einsum("bqe,bde->bqd", query_context_embeddings, sentence_context_embeddings) * mask
        else:
            interaction_matrix = x * mask
        
        if self.context_embedding_dim is not None:
            mask = K.expand_dims(mask) # TODO check the rank of the tensor before expand??
            
            query_context_embeddings = K.dot(query_context_embeddings, self.query_w)
            sentence_context_embeddings = K.dot(sentence_context_embeddings, self.sentence_w)
            
            query_projection_matrix, sentence_projection_matrix = self._query_sentence_vector_to_matrices(query_context_embeddings, sentence_context_embeddings) 
            
            query_projection_matrix = query_projection_matrix * mask
            sentence_projection_matrix = sentence_projection_matrix * mask
            
            interaction_matrix = K.expand_dims(interaction_matrix)
            interaction_matrix = K.concatenate([interaction_matrix, query_projection_matrix, sentence_projection_matrix])
        
        return interaction_matrix



In [10]:
K.clear_session()

input_query = tf.keras.layers.Input((8,), dtype="int32")
input_sentence = tf.keras.layers.Input((4,), dtype="int32")

semantic_interaction = ContextedSemanticInteractions(
                                 context_embedding_layer = [],
                                 cls_token_id = 2.,
                                 sep_token_id = 3.,
                                 pad_token_id = 0.,)

_out = semantic_interaction([input_query, input_sentence])

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()

Tensor("contexted_semantic_interactions/mul:0", shape=(None,), dtype=float32)
Tensor("contexted_semantic_interactions/mul_1:0", shape=(None,), dtype=float32)


TypeError: in converted code:

    <ipython-input-5-5f728536f853>:75 call  *
        query_context_embeddings, sentence_context_embeddings = self._produce_context_embeddings(x[0], x[1])

    TypeError: 'NoneType' object is not iterable
